In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import os
import unicodedata
import re
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [14]:
oba = pd.read_csv('tidy_data_oba.csv')
gwb = pd.read_csv('tidy_data_gwb.csv')
gwb2 = gwb.drop(['date', 'source', 'enc_value'], axis=1)
oba2 = oba.drop(['date', 'source', 'enc_value'], axis=1)
gwb = gwb.drop(['enc_value', 'PCA1', 'PCA2'], axis=1)
oba = oba.drop(['enc_value', 'PCA1', 'PCA2'], axis=1)
columnsTitles = list(oba.columns)
gwb = gwb.reindex(columns=columnsTitles)

<A HREF="https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf">Pandas cheat sheet</A>

In [15]:
oba.describe()

,ADJ,ADP,ADV,AUX,CCONJ,DET,INTJ,NOUN,NUM,PART,...,coleman_liau,dale_chall,gunning_fog,smog,parallel_count,parallel_per_sent,depth_std,words_per_sent_std,syllables_per_sent_std,chars_per_sent_std
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,0.067531,0.068038,0.067378,0.067097,0.067643,0.068023,0.032871,0.068041,0.058334,0.066472,...,9.013232,15.216733,12.835918,12.456703,13.430000,0.111327,2.658715,12.826847,20.896021,72.650070
std,0.004032,0.004444,0.003886,0.004267,0.004120,0.004374,0.025976,0.004533,0.010906,0.004107,...,1.194147,1.866037,1.732366,1.200682,10.364328,0.056000,0.441640,3.580416,5.435917,19.820695
min,0.058444,0.060121,0.059642,0.053903,0.059658,0.060021,0.000000,0.059866,0.000000,0.057288,...,6.452430,11.745237,9.474320,9.994967,1.000000,0.039216,1.602082,4.366539,7.711463,25.319953
25%,0.064596,0.064551,0.064423,0.064082,0.064349,0.064822,0.000000,0.064618,0.056226,0.063562,...,8.223367,14.277198,11.709798,11.607377,6.000000,0.073916,2.439752,10.909153,18.320481,63.253302
50%,0.067778,0.068167,0.067654,0.067164,0.067797,0.068060,0.038543,0.068103,0.060636,0.066798,...,9.029858,15.188358,12.776164,12.452591,10.000000,0.103271,2.613703,12.323611,20.149841,70.548011
75%,0.070712,0.071145,0.070544,0.070578,0.070867,0.071120,0.057884,0.071156,0.064089,0.069083,...,9.739209,15.887708,13.692420,13.116943,17.250000,0.130721,2.790478,14.188336,23.075468,78.259562
max,0.075469,0.084538,0.075100,0.076752,0.078377,0.083426,0.073501,0.085095,0.078067,0.081758,...,12.649953,27.950188,23.152281,18.243606,50.000000,0.400000,5.890671,37.520661,57.954292,209.627765


In [16]:
gwb.describe()

,ADJ,ADP,ADV,AUX,CCONJ,DET,INTJ,NOUN,NUM,PART,...,coleman_liau,dale_chall,gunning_fog,smog,parallel_count,parallel_per_sent,depth_std,words_per_sent_std,syllables_per_sent_std,chars_per_sent_std
count,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,...,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000,83.000000
mean,0.072488,0.072945,0.067060,0.070137,0.071565,0.073274,0.009128,0.073991,0.051417,0.068652,...,10.602322,15.907594,14.345433,13.763908,6.819277,0.109962,2.504680,11.202966,19.563251,66.695169
std,0.005455,0.005465,0.013364,0.006390,0.005900,0.005914,0.019206,0.006801,0.024077,0.009524,...,1.976675,3.488495,3.352043,2.298536,7.151914,0.108668,0.694463,6.567488,11.016983,39.433859
min,0.058520,0.061513,0.003105,0.050562,0.047651,0.061840,0.000000,0.061949,0.000000,0.000000,...,6.805726,10.437930,9.681578,10.117482,0.000000,0.000000,1.272418,5.916080,10.661457,33.079450
25%,0.069099,0.069324,0.066181,0.067160,0.067970,0.069485,0.000000,0.069190,0.045446,0.066600,...,9.396154,13.816470,12.284174,12.335836,1.000000,0.058538,2.068472,8.528752,14.682891,49.687432
50%,0.072099,0.072184,0.069994,0.069864,0.071890,0.072322,0.000000,0.072589,0.060778,0.069652,...,10.250359,15.068501,13.251256,13.122325,4.000000,0.086957,2.362908,9.347886,16.427011,55.372710
75%,0.074720,0.075600,0.072203,0.072398,0.074196,0.076291,0.000683,0.076497,0.067825,0.072600,...,11.511540,17.118907,15.179208,14.212234,12.000000,0.120690,2.733665,12.242672,20.653323,71.204656
max,0.091029,0.090538,0.088203,0.093270,0.090689,0.094317,0.066699,0.096735,0.083895,0.084116,...,17.659618,33.990996,28.514978,21.606579,32.000000,0.615385,5.648916,61.872853,98.841624,357.392874


## Obama vs GWB PCA

In [17]:
combined = pd.concat([oba,gwb], axis=0)
pca_data = combined.drop(['date', 'source'], axis=1)

In [18]:
# Scale data before applying PCA
scaling = StandardScaler()
 
# Use fit and transform method
scaling.fit(pca_data)
scaled_data = scaling.transform(pca_data)

pca = PCA(n_components=2)
pcafit = pca.fit_transform(scaled_data)
 
# Check the dimensions of data after PCA
print(pcafit.shape)

(183, 2)


In [19]:
# check how much variance is explained by each principal component
print(pca.explained_variance_ratio_)

[0.33379843 0.17103792]


In [20]:
fig = px.scatter(pcafit, 
                 x=0, 
                 y=1, 
                 color=combined['source'],
                 hover_name=combined['date'],
                 color_discrete_sequence=px.colors.qualitative.Bold)
fig.show()

In [21]:
loadings = pd.DataFrame(pca.components_.T, columns=['PC1', 'PC2'], index=pca_data.columns)
loadings.sort_values(by=['PC1'], inplace=True)
loadings

,PC1,PC2
num_sents,-0.193044,0.112757
sent_count,-0.193039,0.112746
num_unique_words,-0.189657,0.127659
word_count,-0.186998,0.135574
num_words,-0.186496,0.136110
char_count,-0.182587,0.141216
syl_count,-0.181780,0.141661
dc_word_count,-0.178570,0.145926
parallel_count,-0.167438,0.132381
gf_word_count,-0.165302,0.154509


### Try again without variables relating to length of speech like ['num_sents', 'sent_count', 'word_count']

In [22]:
oba = oba.drop(['num_sents', 'sent_count', 'word_count', 'num_words'], axis=1)
gwb = gwb.drop(['num_sents', 'sent_count', 'word_count', 'num_words'], axis=1)
combined = pd.concat([oba,gwb], axis=0)
pca_data2 = combined.drop(['date', 'source'], axis=1)
scaling2 = StandardScaler()
scaling2.fit(pca_data2)
scaled_data2 = scaling2.transform(pca_data2)
pca2 = PCA(n_components=2)
pcafit2 = pca2.fit_transform(scaled_data2)
print(pca2.explained_variance_ratio_)

[0.31276458 0.17003085]


In [23]:
fig = px.scatter(pcafit2, 
                 x=0, 
                 y=1, 
                 color=combined['source'],
                 hover_name=combined['date'],
                 color_discrete_sequence=px.colors.qualitative.Bold)
fig.show()

In [24]:
pca_data2 = pca_data.drop(['num_sents', 'sent_count', 'word_count', 'num_words'], axis=1)
loadings2 = pd.DataFrame(pca2.components_.T, columns=['PC1', 'PC2'], index=pca_data2.columns)
loadings2.sort_values(by=['PC1'], inplace=True)
loadings2

,PC1,PC2
num_unique_words,-0.176975,0.155811
char_count,-0.166724,0.165815
syl_count,-0.165726,0.165821
INTJ,-0.162473,0.079985
dc_word_count,-0.161689,0.168971
parallel_count,-0.154183,0.159033
gf_word_count,-0.146051,0.172797
poly_word_count,-0.145728,0.173042
SYM,-0.132272,0.123530
surprise,-0.116731,-0.012724


## Obama PCA

In [25]:
oba2 = oba2.drop(['num_sents', 'sent_count', 'word_count', 'num_words'], axis=1)
# Scale data before applying PCA
scaling_oba = StandardScaler()
 
# Use fit and transform method
scaling_oba.fit(oba2)
scaled_data_oba = scaling_oba.transform(oba2)

pca_oba = PCA(n_components=2)
pcafit_oba = pca_oba.fit_transform(scaled_data_oba)
 
# Check the dimensions of data after PCA
print(pcafit_oba.shape)

(100, 2)


In [26]:
# check how much variance is explained by each principal component
print(pca_oba.explained_variance_ratio_)

[0.32957504 0.18912799]


In [27]:
fig_oba = px.scatter(pcafit_oba, 
                 x=0, 
                 y=1, 
                 color=oba['source'],
                 hover_name=oba['date'],
                 color_discrete_sequence=px.colors.qualitative.Bold)
fig_oba.show()

In [28]:
### See contingencyTableAnl.ipynb for list of unexpected words in the left and right speeches

In [29]:
loadings_oba = pd.DataFrame(pca_oba.components_.T, columns=['PC1', 'PC2'], index=oba2.columns)
loadings_oba.sort_values(by=['PC1'], inplace=True)
loadings_oba

,PC1,PC2
INTJ,-0.165296,0.046585
SYM,-0.148049,0.088380
PCA2,-0.137298,-0.123400
num_unique_words,-0.128090,0.172443
PCA1,-0.128019,-0.109164
char_count,-0.120576,0.183928
syl_count,-0.119044,0.184970
dc_word_count,-0.118034,0.186266
joy,-0.116373,-0.044636
parallel_count,-0.102373,0.171144


### GWB PCA plot

In [30]:
gwb2 = gwb2.drop(['num_sents', 'sent_count', 'word_count', 'num_words'], axis=1)
# Scale data before applying PCA
scaling_gwb = StandardScaler()
 
# Use fit and transform method
scaling_gwb.fit(gwb2)
scaled_data_gwb = scaling_gwb.transform(gwb2)

pca_gwb = PCA(n_components=2)
pcafit_gwb = pca_gwb.fit_transform(scaled_data_gwb)
 
# Check the dimensions of data after PCA
print(pcafit_gwb.shape)

(83, 2)


In [31]:
# check how much variance is explained by each principal component
print(pca_gwb.explained_variance_ratio_)

[0.30821975 0.17151709]


In [32]:
fig_gwb = px.scatter(pcafit_gwb, 
                 x=0, 
                 y=1, 
                 color=gwb['source'],
                 hover_name=gwb['date'],
                 color_discrete_sequence=px.colors.qualitative.Bold)
fig_gwb.show()

In [33]:
loadings_gwb = pd.DataFrame(pca_gwb.components_.T, columns=['PC1', 'PC2'], index=gwb2.columns)
loadings_gwb.sort_values(by=['PC1'], inplace=True)
loadings_gwb

,PC1,PC2
num_unique_words,-0.167634,-0.190771
char_count,-0.159654,-0.191419
syl_count,-0.159423,-0.191841
parallel_count,-0.153623,-0.173255
dc_word_count,-0.153091,-0.199037
gf_word_count,-0.143819,-0.209893
poly_word_count,-0.143683,-0.209901
surprise,-0.121386,-0.014973
INTJ,-0.107468,-0.058196
anger,-0.098511,0.007479
